In [53]:
import pandas as pd
import numpy as np

In [54]:
df = pd.read_csv('../data/clean_multi_stock_data.csv')
df.rename(columns={'Price':'Date'},inplace=True)
df.Date = pd.to_datetime(df["Date"], errors="coerce")
df.head()

,Date,Close,High,Low,Open,Volume,Ticker,Target
0,2015-01-02,24.261053,24.729276,23.821677,24.718180,212818400,AAPL,0
1,2015-01-05,23.577576,24.110152,23.391175,24.030265,257142000,AAPL,1
2,2015-01-06,23.579796,23.839426,23.218087,23.641929,263188400,AAPL,1
3,2015-01-07,23.910433,24.010290,23.677430,23.788384,160423600,AAPL,1
4,2015-01-08,24.829126,24.886823,24.121244,24.238856,237458000,AAPL,1


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10900 entries, 0 to 10899
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    10900 non-null  datetime64[ns]
 1   Close   10900 non-null  float64       
 2   High    10900 non-null  float64       
 3   Low     10900 non-null  float64       
 4   Open    10900 non-null  float64       
 5   Volume  10900 non-null  int64         
 6   Ticker  10900 non-null  object        
 7   Target  10900 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(2), object(1)
memory usage: 681.4+ KB


In [56]:
df.corr(numeric_only=True)

,Close,High,Low,Open,Volume,Target
Close,1.000000,0.999775,0.999789,0.999581,-0.430323,-0.007098
High,0.999775,1.000000,0.999632,0.999799,-0.428615,-0.006763
Low,0.999789,0.999632,1.000000,0.999760,-0.432090,-0.006681
Open,0.999581,0.999799,0.999760,1.000000,-0.430185,-0.006588
Volume,-0.430323,-0.428615,-0.432090,-0.430185,1.000000,0.014341
Target,-0.007098,-0.006763,-0.006681,-0.006588,0.014341,1.000000


In [57]:
df['RAV'] = df.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(7).mean())
df['volatility'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(20).std())

In [58]:
df["Buy_Sell_Strength"] = (df["Close"]-df["Low"]) / (df["High"] - df["Low"])
df["Weighted_Strength"] = (df["Buy_Sell_Strength"] - 0.5) * (df["Volume"]/ df["RAV"])
df["Trend"] = (df["Buy_Sell_Strength"] - 0.5).rolling(25).mean()
df["Returns"] = df["Close"].pct_change()
df["Log_returns"] = (df["Close"] / df["Close"].shift(1).apply(np.log))
df.dropna(inplace=True)

In [59]:
df.drop(["High","Low","Open"],axis="columns",inplace=True)
df.head()

,Date,Close,Volume,Ticker,Target,RAV,volatility,Buy_Sell_Strength,Weighted_Strength,Trend,Returns,Log_returns
24,2015-02-06,26.495508,174826400,AAPL,1,2.508224e+08,1.046604,0.266668,-0.162635,-0.007055,-0.008421,8.064544
25,2015-02-09,26.671497,155559200,AAPL,1,2.247958e+08,1.096022,0.914897,0.287110,0.010177,0.006642,8.139060
26,2015-02-10,27.183907,248034000,AAPL,1,2.123746e+08,1.149111,0.934671,0.507655,0.037193,0.019212,8.278701
27,2015-02-11,27.821062,294247200,AAPL,1,2.185590e+08,1.245724,0.983471,0.650899,0.053246,0.023439,8.423924
28,2015-02-12,28.173058,297898000,AAPL,1,2.314498e+08,1.339347,0.465967,-0.043803,0.043885,0.012652,8.471079


In [60]:
df_dummies = pd.get_dummies(data=df["Ticker"])
df_dummies.head()

,AAPL,MSFT,NVDA,TSLA
24,True,False,False,False
25,True,False,False,False
26,True,False,False,False
27,True,False,False,False
28,True,False,False,False


In [61]:
df = pd.concat([df,df_dummies],axis=1)
df.drop(['Ticker'],axis="columns",inplace=True)
df.head()

,Date,Close,Volume,Target,RAV,volatility,Buy_Sell_Strength,Weighted_Strength,Trend,Returns,Log_returns,AAPL,MSFT,NVDA,TSLA
24,2015-02-06,26.495508,174826400,1,2.508224e+08,1.046604,0.266668,-0.162635,-0.007055,-0.008421,8.064544,True,False,False,False
25,2015-02-09,26.671497,155559200,1,2.247958e+08,1.096022,0.914897,0.287110,0.010177,0.006642,8.139060,True,False,False,False
26,2015-02-10,27.183907,248034000,1,2.123746e+08,1.149111,0.934671,0.507655,0.037193,0.019212,8.278701,True,False,False,False
27,2015-02-11,27.821062,294247200,1,2.185590e+08,1.245724,0.983471,0.650899,0.053246,0.023439,8.423924,True,False,False,False
28,2015-02-12,28.173058,297898000,1,2.314498e+08,1.339347,0.465967,-0.043803,0.043885,0.012652,8.471079,True,False,False,False


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10819 entries, 24 to 10899
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               10819 non-null  datetime64[ns]
 1   Close              10819 non-null  float64       
 2   Volume             10819 non-null  int64         
 3   Target             10819 non-null  int64         
 4   RAV                10819 non-null  float64       
 5   volatility         10819 non-null  float64       
 6   Buy_Sell_Strength  10819 non-null  float64       
 7   Weighted_Strength  10819 non-null  float64       
 8   Trend              10819 non-null  float64       
 9   Returns            10819 non-null  float64       
 10  Log_returns        10819 non-null  float64       
 11  AAPL               10819 non-null  bool          
 12  MSFT               10819 non-null  bool          
 13  NVDA               10819 non-null  bool          
 14  TSLA      

In [63]:
df.to_csv("../data/engineered_features_multi_stock_data.csv")

In [64]:
df_AAPL = pd.read_csv("../data/clean_AAPL_stock_data.csv")
df_MSFT = pd.read_csv("../data/clean_MSFT_stock_data.csv")
df_NVDA = pd.read_csv("../data/clean_NVDA_stock_data.csv")
df_TSLA = pd.read_csv("../data/clean_TSLA_stock_data.csv")

datasets = [df_AAPL,df_MSFT,df_NVDA,df_TSLA]

for df in datasets:
    df.rename(columns={'Price':'Date'},inplace=True)
    df.Date = pd.to_datetime(df["Date"], errors="coerce")
    print(df.head(2))

        Date      Close       High        Low       Open     Volume Ticker  \
0 2015-01-02  24.261053  24.729276  23.821677  24.718180  212818400   AAPL   
1 2015-01-05  23.577576  24.110152  23.391175  24.030265  257142000   AAPL   

   Target  
0       0  
1       1  
        Date      Close       High        Low       Open    Volume Ticker  \
0 2015-01-02  39.933044  40.496684  39.745167  39.847646  27913900   MSFT   
1 2015-01-05  39.565834  39.907432  39.497512  39.599992  39673900   MSFT   

   Target  
0       0  
1       0  
        Date     Close      High      Low      Open     Volume Ticker  Target
0 2015-01-02  0.483038  0.486638  0.47536  0.483038  113680000   NVDA       0
1 2015-01-05  0.474880  0.484478  0.47272  0.483038  197952000   NVDA       0
        Date      Close       High        Low       Open    Volume Ticker  \
0 2015-01-02  14.620667  14.883333  14.217333  14.858000  71466000   TSLA   
1 2015-01-05  14.006000  14.433333  13.810667  14.303333  80527500   TSLA

In [65]:
for df in datasets:
    df['RAV'] = df.groupby('Ticker')['Volume'].transform(lambda x: x.rolling(7).mean())
    df['volatility'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(20).std())
    df["Buy_Sell_Strength"] = (df["Close"]-df["Low"]) / (df["High"] - df["Low"])
    df["Weighted_Strength"] = (df["Buy_Sell_Strength"] - 0.5) * (df["Volume"]/ df["RAV"])
    df["Trend"] = (df["Buy_Sell_Strength"] - 0.5).rolling(25).mean()
    df["Returns"] = df["Close"].pct_change()
    df["Log_returns"] = (df["Close"] / df["Close"].shift(1).apply(np.log))
    df.dropna(inplace=True)
    df.drop(["High","Low","Open"],axis="columns",inplace=True)

In [66]:
df_AAPL.head()

,Date,Close,Volume,Ticker,Target,RAV,volatility,Buy_Sell_Strength,Weighted_Strength,Trend,Returns,Log_returns
24,2015-02-06,26.495508,174826400,AAPL,1,2.508224e+08,1.046604,0.266668,-0.162635,-0.007055,-0.008421,8.064544
25,2015-02-09,26.671497,155559200,AAPL,1,2.247958e+08,1.096022,0.914897,0.287110,0.010177,0.006642,8.139060
26,2015-02-10,27.183907,248034000,AAPL,1,2.123746e+08,1.149111,0.934671,0.507655,0.037193,0.019212,8.278701
27,2015-02-11,27.821062,294247200,AAPL,1,2.185590e+08,1.245724,0.983471,0.650899,0.053246,0.023439,8.423924
28,2015-02-12,28.173058,297898000,AAPL,1,2.314498e+08,1.339347,0.465967,-0.043803,0.043885,0.012652,8.471079


In [67]:
interleaved_df = pd.concat(datasets).sort_index(kind='merge').reset_index(drop=True)
interleaved_df.head(10)

,Date,Close,Volume,Ticker,Target,RAV,volatility,Buy_Sell_Strength,Weighted_Strength,Trend,Returns,Log_returns
0,2015-02-06,26.495508,174826400,AAPL,1,2.508224e+08,1.046604,0.266668,-0.162635,-0.007055,-0.008421,8.064544
1,2015-02-06,36.218159,34616600,MSFT,0,5.097210e+07,2.133874,0.406248,-0.063670,0.022419,-0.000942,10.087210
2,2015-02-06,0.489517,210524000,NVDA,0,2.155857e+08,0.010619,0.225000,-0.268543,0.023777,-0.004392,-0.689525
3,2015-02-06,14.490667,48658500,TSLA,1,5.486293e+07,0.608197,0.124639,-0.332912,0.016009,-0.016426,5.386730
4,2015-02-09,26.671497,155559200,AAPL,1,2.247958e+08,1.096022,0.914897,0.287110,0.010177,0.006642,8.139060
5,2015-02-09,36.175453,31381100,MSFT,1,4.637150e+07,2.087252,0.283020,-0.146837,0.023740,-0.001179,10.077961
6,2015-02-09,0.489037,273944000,NVDA,1,2.211006e+08,0.010830,0.793109,0.363162,0.028268,-0.000980,-0.684604
7,2015-02-09,14.498667,52086000,TSLA,0,5.470071e+07,0.638038,0.924241,0.403962,0.028755,0.000552,5.423094
8,2015-02-10,27.183907,248034000,AAPL,1,2.123746e+08,1.149111,0.934671,0.507655,0.037193,0.019212,8.278701
9,2015-02-10,36.380413,29670700,MSFT,0,3.946661e+07,2.044404,0.711861,0.159276,0.045548,0.005666,10.138393


In [68]:
df_dummies = pd.get_dummies(data=interleaved_df["Ticker"])
df_dummies.head()

,AAPL,MSFT,NVDA,TSLA
0,True,False,False,False
1,False,True,False,False
2,False,False,True,False
3,False,False,False,True
4,True,False,False,False


In [69]:
df_2 = pd.concat([interleaved_df,df_dummies],axis=1)
df_2.drop(['Ticker'],axis="columns",inplace=True)
df_2.head()

,Date,Close,Volume,Target,RAV,volatility,Buy_Sell_Strength,Weighted_Strength,Trend,Returns,Log_returns,AAPL,MSFT,NVDA,TSLA
0,2015-02-06,26.495508,174826400,1,2.508224e+08,1.046604,0.266668,-0.162635,-0.007055,-0.008421,8.064544,True,False,False,False
1,2015-02-06,36.218159,34616600,0,5.097210e+07,2.133874,0.406248,-0.063670,0.022419,-0.000942,10.087210,False,True,False,False
2,2015-02-06,0.489517,210524000,0,2.155857e+08,0.010619,0.225000,-0.268543,0.023777,-0.004392,-0.689525,False,False,True,False
3,2015-02-06,14.490667,48658500,1,5.486293e+07,0.608197,0.124639,-0.332912,0.016009,-0.016426,5.386730,False,False,False,True
4,2015-02-09,26.671497,155559200,1,2.247958e+08,1.096022,0.914897,0.287110,0.010177,0.006642,8.139060,True,False,False,False


In [70]:
df_2['market_return'] = df_2.groupby('Date')['Returns'].transform('mean')
df_2['rel_return'] = df_2['Returns'] - df_2['market_return']
df_2['mean_return_others'] = df_2.groupby('Date')['Returns'].transform(lambda x: x.mean())
df_2['divergence'] = (df_2['Returns'] - df_2['mean_return_others']).abs()
df_2['volume_rank'] = df_2.groupby('Date')['Volume'].rank(pct=True)
df_2['return_rank'] = df_2.groupby('Date')['Returns'].rank(pct=True)
df_2['lag_market_return'] = df_2.groupby('Date')['market_return'].shift(1)
df_2['market_std'] = df_2.groupby('Date')['Returns'].transform('std')
df_2['zscore_vs_market'] = (df_2['Returns'] - df_2['market_return']) / df_2['market_std']
df_2['return_1'] = df_2.groupby('Date')['Returns'].shift(1)
df_2['return_2'] = df_2.groupby('Date')['Returns'].shift(2)
df_2['vol_3'] = df_2['volatility'].rolling(3).mean()
df_2['close_z'] = (df_2['Close'] - df_2['Close'].rolling(10).mean()) / df_2['Close'].rolling(10).std()
df_2['market_mean_return'] = df_2.groupby('Date')['Returns'].transform('mean')

In [88]:
df_2.to_csv("../data/engineered_interleaved_features_multi_stock_data.csv")
import os
os.makedirs("../Version 3/data", exist_ok=True)
df_2.to_csv(r"C:\Users\dalu\OneDrive\Documents\stock_predictor\Version 3/data/engineered_interleaved_features_multi_stock_data.csv")